# meshing of solids
In this example, I mesh a 3d imported geometry. A circle is etched on a surface of the solid. The OpenCascade kernel (OCC) is selected to operate on imported geometries read from *.stp files. I cannot do the same with the natural kernel of GMSH.

In [1]:
import numpy as np
import gmsh
import sys
import os
import math

def gmsh_plot():

    # Synchronize the model to build the geometry
    gmsh.model.occ.synchronize()

    # visualize the model, whatever you do here it is avaialble later in the script
    if '-nopopup' not in sys.argv:
        gmsh.fltk.run()

# Initialize the Gmsh API
gmsh.initialize()
gmsh.model.add("t20")
#gmsh.merge("two_blocks.stp")
gmsh.model.occ.importShapes("two_blocks.stp")

# Synchronize the CAD model with the Gmsh model
gmsh.model.occ.synchronize()

# remove matching entities
gmsh.model.occ.remove_all_duplicates()

In [ ]:
# Get all points
point_tags = gmsh.model.getEntities(0)  # 0 refers to points
line_tags = gmsh.model.getEntities(1)  # 0 refers to points
surface_tags = gmsh.model.getEntities(2)  # 0 refers to points
volume_tags = gmsh.model.getEntities(3)  # 0 refers to points

for point_tag in point_tags:
    point_coord = gmsh.model.get_value(0, point_tag[1], [0])
    print(f"Point ID: {point_tag[1]}, coordinates: {point_coord}")

for line_tag in line_tags:
    # parametrization bounds of the line
    bounds = gmsh.model.get_parametrization_bounds(1, line_tag[1])
    # coordinate of the mid-point of each line
    mid_point_coord = gmsh.model.get_value(1, line_tag[1], (bounds[0]+bounds[1])/2)
    print(f"Point ID: {line_tag[1]}, mid coordinates: {mid_point_coord}")


In [5]:
gmsh_plot()

In [ ]:
point_coord1 = gmsh.model.get_value(0, 15, [0])
point_coord2 = gmsh.model.get_value(0, 16, [0])
point_coord3 = gmsh.model.get_value(0, 17, [0])
point_coord4 = gmsh.model.get_value(0, 18, [0])

print(f"Point 1: {point_coord1}")
print(f"Point 2: {point_coord2}")
print(f"Point 3: {point_coord3}")
print(f"Point 4: {point_coord4}")

center_coord_x = (point_coord1[0]+point_coord2[0]+point_coord3[0]+point_coord4[0])/4
center_coord_y = (point_coord1[1]+point_coord2[1]+point_coord3[1]+point_coord4[1])/4
center_coord_z = (point_coord1[2]+point_coord2[2]+point_coord3[2]+point_coord4[2])/4

print(f"Point x: {center_coord_x}")
print(f"Point y: {center_coord_y}")
print(f"Point z: {center_coord_z}")

r = 2.0

# x (double), y (double), z (double), meshSize = 0. (double), tag = -1 (integer)
gmsh.model.occ.add_point(center_coord_x, center_coord_y, center_coord_z, 1, 100)
gmsh.model.occ.add_point(center_coord_x+r, center_coord_y, center_coord_z, 1, 101)
gmsh.model.occ.add_point(center_coord_x, center_coord_y, center_coord_z+r, 1, 102)
gmsh.model.occ.add_point(center_coord_x-r, center_coord_y, center_coord_z, 1, 103)
gmsh.model.occ.add_point(center_coord_x, center_coord_y, center_coord_z-r, 1, 104)

# startTag (integer), centerTag (integer), endTag (integer), tag = -1 (integer), nx = 0. (double), ny = 0. (double), nz = 0. (double)
# startTag (integer), middleTag (integer), endTag (integer), tag = -1 (integer), center = True (boolean)
gmsh.model.occ.add_circle_arc(101,102,103,201,False)#,0,1,0)
gmsh.model.occ.add_circle_arc(103,104,101,202,False)#,0,1,0)
#gmsh.model.occ.add_circle_arc(103,100,104,203,0,1,0)
#gmsh.model.occ.add_circle_arc(104,100,101,204,0,1,0)

# remove matching entities
gmsh.model.occ.remove_all_duplicates()

# We have to synchronize before embedding entites:
gmsh.model.occ.synchronize()

In [7]:
# One can force this point to be included ("embedded") in the 2D mesh, using the
# `embed()' function:
#gmsh.model.mesh.embed(1,[201,202,203,204],2,14)
gmsh.model.mesh.embed(1,[201,202],2,14)

# We have to synchronize before embedding entites:
gmsh.model.occ.synchronize()

In [6]:
gmsh_plot()

In [ ]:
#gmsh.model.add_physical_group(3, [1],name="my concave volume")
#gmsh.model.add_physical_group(3, [2],name="my cube")
#gmsh.model.get_boundary([(3,1)], True, True, False)
#gmsh.model.get_boundary([(3,2)], True, True, False)
#gmsh.model.get_entities(3)

In [8]:
# Finally, let's specify a global mesh size and mesh the partitioned model:
gmsh.option.set_number("Mesh.MeshSizeMin", 0.1)
gmsh.option.set_number("Mesh.MeshSizeMax", 0.2)
gmsh.model.mesh.generate(2)

In [9]:
gmsh_plot()

In [10]:
# We have to synchronize before embedding entites:
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)

In [11]:
gmsh_plot()